# Machine learning

## EXERCISE: Simple linear regression from scratch

[Adapted from Data Science from Scratch, Ch. 14.]

For this exercise, we'll build simple linear regression from scratch following the text book.

We'll use the data from the book's imaginary social network (DataSciencester):

* `num_friends` is the friend count for each user.
* `daily_minutes` is the average time spend per day on the site for each user.

In [ ]:
num_friends = [100,49,41,40,25,21,21,19,19,18,18,16,15,15,15,15,14,14,13,13,13,13,12,12,11,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
daily_minutes = [1,68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

### Preliminary maths functions

First we'll define some linear algebra and stats functions (from Ch. 4&5).

In [ ]:
import math

####
#
# LINEAR ALGEBRA
#
####

def vector_subtract(v, w):
    """subtracts two vectors componentwise"""
    return [v_i - w_i for v_i, w_i in zip(v,w)]

def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

####
#
# STATS
#
####

def mean(x):
    return sum(x) / len(x)

def de_mean(x):
    """translate x by subtracting its mean (so the result has mean 0)"""
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

def variance(x):
    """assumes x has at least two elements"""
    n = len(x)
    deviations = de_mean(x)
    return sum_of_squares(deviations) / (n - 1)

def standard_deviation(x):
    return math.sqrt(variance(x))

####
#
# CORRELATION
#
#####

def covariance(x, y):
    n = len(x)
    return dot(de_mean(x), de_mean(y)) / (n - 1)

def correlation(x, y):
    stdev_x = standard_deviation(x)
    stdev_y = standard_deviation(y)
    if stdev_x > 0 and stdev_y > 0:
        return covariance(x, y) / stdev_x / stdev_y
    else:
        return 0 # if no variation, correlation is zero

### TODO Compare to numpy/scipy

- Compare our functions above to the `mean`, `var` and `std` functions from `numpy`. Are the results effectively the same?
- Compare the correlation above to the `pearsonr` function from `scipy.stats`. Are the results effectively the same?

In [ ]:
# 1 - Mostly the same with some small tolerable differences.
import numpy as np
print('\nComparing stats for number of friends:')
print('-'*47)
print('{:15} {:>15} {:>15}'.format('Statistic', 'From Numpy', 'From Scratch'))
print('-'*47)
for np_fn, sc_fn in [(np.mean, mean), (np.var, variance), (np.std, standard_deviation)]:
    print('{:15} {:15.2f} {:15.2f}'.format(np_fn.__name__, np_fn(num_friends), sc_fn(num_friends)))
print('-'*47)

# 2 - 
from scipy.stats import pearsonr
print('\nCorrelation from scipy: {:.3f}'.format(pearsonr(num_friends, daily_minutes)[0]))
print('Correlation from scratch: {:.3f}'.format(correlation(num_friends, daily_minutes)))


### Removing outliers

Let's have a look at the data.

The data point for somebody who has 100 friends but spends 1 minute per day looks like an outlier. Let's remove it (see pages 64-65 of the text book for a discussion).

This leads to much stronger correlation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

print('\nScatter plot for daily minutes vs number of friends:')
_ = plt.scatter(num_friends, daily_minutes)
_ = plt.xlim(0,100)
_ = plt.ylim(0,100)
_ = plt.xlabel('Number of friends')
_ = plt.ylabel('Average minutes per day on site')

outlier = num_friends.index(100) # index of outlier

num_friends_good = [x
                    for i, x in enumerate(num_friends)
                    if i != outlier]

daily_minutes_good = [x
                      for i, x in enumerate(daily_minutes)
                      if i != outlier]

In [ ]:
print('Correlation before outlier removal: {:.3f}'.format(correlation(num_friends, daily_minutes)))
print('Correlation after outlier removal: {:.3f}'.format(correlation(num_friends_good, daily_minutes_good)))

### Simple linear regression

That's the preliminaries all done. Now we can look at linear regression between two variables.

Let's assume that we think having more friends causes people to spend more time on the site (see pages 65-67 of the book for a good discussion of caveats and other possible explanations).

We're now going to build a simple linear model to describe this relationship.

In particular, we assume a model of the form:

`y_i = beta * x_i + alpha`

Note this takes us right back to [linear equations in algebra](https://en.wikipedia.org/wiki/Linear_equation):

`y = mx + b`

Looking back at the scatter plot and correlation, this doesn't seems plausible if we assume some noise.

Let's see how this works for our data.

In [ ]:
def predict(alpha, beta, x_i):
    """the linear model
    alpha and beta are our model parameters
    x_i is the data point
    return the predicted y value for x_i"""
    return beta * x_i + alpha

def error(alpha, beta, x_i, y_i):
    """the difference between the true y_i
    and our predicted y value"""
    return y_i - predict(alpha, beta, x_i)

def sum_of_squared_errors(alpha, beta, x, y):
    return sum(error(alpha, beta, x_i, y_i) ** 2
               for x_i, y_i in zip(x, y))

def least_squares_fit(x,y):
    """given training values for x and y,
    find the least-squares values of alpha and beta"""
    beta = correlation(x, y) * standard_deviation(y) / standard_deviation(x)
    alpha = mean(y) - beta * mean(x)
    return alpha, beta

def total_sum_of_squares(y):
    """the total squared variation of y_i's from their mean"""
    return sum(v ** 2 for v in de_mean(y))

def r_squared(alpha, beta, x, y):
    """the fraction of variation in y captured by the model, which equals
    1 - the fraction of variation in y not captured by the model"""
    return 1.0 - (sum_of_squared_errors(alpha, beta, x, y) /
                  total_sum_of_squares(y))

alpha, beta = least_squares_fit(num_friends_good, daily_minutes_good)
print("alpha", alpha)
print("beta", beta)

r_squared_value = r_squared(alpha, beta, num_friends_good, daily_minutes_good)
print("r-squared", r_squared_value)

### TODO Assessing fit and precision

R-squared gives an indication of [how well the model fits our data](http://blog.minitab.com/blog/adventures-in-statistics/regression-analysis-how-do-i-interpret-r-squared-and-assess-the-goodness-of-fit). It ranges from 0 to 1, with higher values indicating a better fit.

Standard error is defined as [the square root of the sum of squared errors divided by N](http://onlinestatbook.com/2/regression/accuracy.html). 

As a [rule of thumb](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule), we can calculate a 95% confidence interval as [+/- 2 * standard_error from the regression line](http://blog.minitab.com/blog/adventures-in-statistics/regression-analysis-how-to-interpret-s-the-standard-error-of-the-regression).

- What r-squared for our model above? Does this indicate a good fit?
- Define a `standard_error` function using `sum_of_squared_errors` and `math.sqrt`. What is the standard error of our model?
- Redraw the scatterplot from above. This time use `plt.plot` to overlay the regression line and the 95% prediction interval at + and - 2 times the calculated standard error.
- Suppose our requirement is that predictions should be +/- 10 minutes of the actual value. Can we say this is with 95% confidence?

In [ ]:
# 1 - The value here is 0.329. This suggests that our model only partly explains the data
#     so there must be other factors at play (see page 176 of the text book).


# 2 - 
def standard_error(alpha, beta, x, y):
    return math.sqrt(sum_of_squared_errors(alpha, beta, x, y) / len(y))

stderr = standard_error(alpha, beta, num_friends_good, daily_minutes_good)
print("standard error", stderr)

# 3 - 
print('\nScatter plot with regression line and 95% prediction interval:')
_ = plt.scatter(num_friends_good, daily_minutes_good)
_ = plt.xlim(0,100)
_ = plt.ylim(0,100)
_ = plt.xlabel('Number of friends')
_ = plt.ylabel('Average minutes per day on site')
_ = plt.plot([0,100], [predict(alpha,beta,0), predict(alpha,beta,100)], 'r',
             [0,100], [predict(alpha,beta,0)-2*stderr, predict(alpha,beta,100)-2*stderr], 'r--',
             [0,100], [predict(alpha,beta,0)+2*stderr, predict(alpha,beta,100)+2*stderr], 'r--')

# 4 - Nope. The standard error is ~8. 2 * 8 > 10.

### TODO Compare to scipy

- Compare our result to the `linregress` function from `scipy.stats`. Are alpha, beta and r-squared the same?
- The `scipy` implementation also returns a p-value for the H0 that slope is 0. What is the p-value? Can we reject the H0 at p<=0.01?

Note the scipy implementation returns a standard error, but it the error for the slope coefficient instead of the fitting error.

In [ ]:
# 1 - 
from scipy.stats import linregress
slope, intercept, r_value, p_value, slope_stderr = linregress(num_friends_good, daily_minutes_good)

print('\nComparing linear regression values:')
print('-'*47)
print('{:15} {:>15} {:>15}'.format('Statistic', 'From Scipy', 'From Scratch'))
print('-'*47)
for name, sc_val, sp_val in [("alpha", intercept, alpha),
                             ("beta", slope, beta),
                             ("r-squared", r_squared_value, r_value**2)]:
    print('{:15} {:15.2f} {:15.2f}'.format(name, sc_val, sp_val))
print('-'*47)

# 2 - 
print('\np-value for a two-sided test of H0 that slope is 0: {}'.format(p_value))
print('Can we reject H0?', 'Yes' if p_value<=0.01 else 'No')

## *STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.*

## EXERCISE: Multiple regression



Let's at the [Boston Housing data set](https://archive.ics.uci.edu/ml/datasets/Housing). This contains information about housing values by suburb and related information.

### Loading and visualising data

Let's load the data and have a quick look at some descriptive stats for our features.

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.DESCR)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def cols_from_rows(data):
    cols = [[] for _ in data[0]] # init with empty list for each column
    for row in data:
        for i, val in enumerate(row):
            cols[i].append(val)
    return cols

print('Descriptive stats for feature values:')
cols = cols_from_rows(boston.data)
print('-'*65)
print('{:10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format('Name', 'Min', 'Max', 'Mean', 'Stdev', 'Median'))
print('-'*65)
for i, col in enumerate(cols):
    print('{:10} {:10.2f} {:10.2f} {:10.2f} {:10.2f} {:10.2f}'\
          .format(boston.feature_names[i], min(col), max(col), np.mean(col), np.std(col), np.median(col)))
print('-'*65)
    
print('\nBoxplots of feature values:')
cols = cols_from_rows(boston.data) + [boston.target]
_ = plt.boxplot(cols, vert=False, notch=True, flierprops={'marker':'.'})
labels = list(boston.feature_names) + ['VALUE']
_ = plt.yticks(range(1,15), labels)

### Linear regression in scikit-learn

Now let's fit a linear model for predicting a suburbs median house value given the other features.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import math
import numpy as np

# First let's create a train and test split
X_train, X_test, Y_train, Y_test = train_test_split(boston.data, boston.target, test_size=0.33,
                                                    random_state=5) # so we get the same results

# Now let's fit a model
lm = LinearRegression()
_ = lm.fit(X_train, Y_train)

print('Intercept:', lm.intercept_)
print('Coefficients:\n', lm.coef_)
print('\nR-squared:', lm.score(X_train, Y_train))

# We use the score method to get r-squared
print('\nR-squared:', lm.score(X_train, Y_train))

# We can predict the median price for a new neighbourhoods
print('\nPredicted price of first five neighbourhoods from test split:\n', lm.predict(X_test)[:5])

# We use the score method to get r-squared
print('\nR-squared:', lm.score(X_train, Y_train))

# We can also calculate the standard error
stderr = math.sqrt(np.mean((Y_train - lm.predict(X_train))**2))
print('\nStandard error:', stderr)

### TODO Evaluating predictions 

- Does r_squared indicate a good fit?
- Use `mean_squared_error` from `sklearn.metrics` to calculate standard error. Compare this to the value we calculated in the previous cell.
- According to the 95% prediction interval, how close will our predictions be to the actual value? What if we calculate over the test data instead?

[Residual plots](http://docs.statwing.com/interpreting-residual-plots-to-improve-your-regression/) scatter residuals (y_true - y_predicted) on the y-axis versus predicted values on the x-axis. A residual plot indicates that a linear model is appropriate if the points are randomly distributed around the `y=0` line.

- Draw a residual plot with training data in blue and test data in green. Is a linear model appropriate for our data? Is prediction performance comparable to training performance?

In [ ]:
# 1 - Yes, r_squared indicates that our model explains the data reasonable well.
#     But we should look at standard error as well (page 183 of Data Science from Scratch).

# 2 - 
from sklearn.metrics import mean_squared_error
se1 = math.sqrt(np.mean((Y_train - lm.predict(X_train))**2))
print('Standard error:', se1)
se2 = math.sqrt(mean_squared_error(Y_train, lm.predict(X_train)))
print('Standard error using sklearn.metrics:', se2)

# 3 - Note we have a fairly small data set (339 in training, 167 in test).
#     So this value will vary depending on our split. 
print('\nPredictions should be within {:.1f}k dollars at 95% confidence according to training data.'.format(2*se2))
se_test = math.sqrt(mean_squared_error(Y_test, lm.predict(X_test)))
print('Predictions should be within {:.1f}k dollars at 95% confidence according to test data.'.format(2*se_test))

# 4 - Yes the linear model is appropriate
print('\nResidual plot for training data (blue) and test data (green):')
_ = plt.scatter(lm.predict(X_train), Y_train-lm.predict(X_train), c='blue', s=40, alpha=0.5, edgecolor='white')
_ = plt.scatter(lm.predict(X_test), Y_test-lm.predict(X_test), c='green', s=40, alpha=0.5, edgecolor='white')
_ = plt.plot([-10,50], [0,0], c='black')
_ = plt.ylabel('Residuals ($y - \hat{y}$)')
_ = plt.xlabel('Predicted values ($\hat{y}$)')

## *STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.*

## EXERCISE: Logistic regression



We'll use the Iris data set. Like the digits data from week 7, this is another classic machine learning data set.

### Loading and visualising data

Let's load the data and have a quick look at some descriptive stats for our features.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
print(iris.DESCR)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def cols_from_rows(data):
    cols = [[] for _ in data[0]] # init with empty list for each column
    for row in data:
        for i, val in enumerate(row):
            cols[i].append(val)
    return cols

print('Descriptive stats for feature values:')
cols = cols_from_rows(iris.data)
print('-'*75)
print('{:20} {:>10} {:>10} {:>10} {:>10} {:>10}'.format('Name', 'Min', 'Max', 'Mean', 'Stdev', 'Median'))
print('-'*75)
for i, col in enumerate(cols):
    print('{:20} {:10.2f} {:10.2f} {:10.2f} {:10.2f} {:10.2f}'\
          .format(iris.feature_names[i], min(col), max(col), np.mean(col), np.std(col), np.median(col)))
print('-'*75)
    
print('\nBoxplots of feature values:')
_ = plt.boxplot(cols_from_rows(iris.data), vert=False, notch=True, flierprops={'marker':'.'})
_ = plt.yticks(range(1,5), iris.feature_names)

### Logistic regression in scikit-learn

Now let's use logistic regression to learn a model of iris type given sepal and petal measurements.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# First let's create a train and test split
X_train, X_test, Y_train, Y_test = train_test_split(iris.data, iris.target, test_size=0.33,
                                                    random_state=5) # so we get the same results

# Now let's fit a model
logreg = LogisticRegression()
_ = logreg.fit(X_train, Y_train)
print('Intercept:', logreg.intercept_)
print('Coefficients:\n', logreg.coef_)

# We can predict the type of new organisms given measurements
print('\nPredicted type of first five organisms from test split:', logreg.predict(X_test)[:5])
print('Actual type of first five organisms from test split:', Y_test[:5])

### Evaluating classification

Recall from week 7 that `sklearn.metrics` includes various evaluation measures. 

In [ ]:
from sklearn.metrics import classification_report
key=', '.join(['{}={}'.format(i,name) for i,name in enumerate(iris.target_names)])
print('Classification report ({}):\n'.format(key))
print(classification_report(Y_test, logreg.predict(X_test)))

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

print('Confusion matrix ({}):\n'.format(key))
_ = plt.matshow(confusion_matrix(Y_test, logreg.predict(X_test)), cmap=plt.cm.binary, interpolation='nearest')
_ = plt.colorbar()
_ = plt.ylabel('true label')
_ = plt.xlabel('predicted label')

### TODO Choose parameters using grid search


- Choose the best C values in [1, 10, 100, 1000, 10000, 100000, 1000000] and the best penalty in ['l1', 'l2']
- Which is the best C? Which is the best penalty?
- How do precision, recall and f1 scores compare?

In [ ]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

param_grid = [
    {'C': [1, 10, 100, 1000, 1e4, 1e5, 1e6, 1e7], 'penalty': ['l1', 'l2']}
]
logreg = GridSearchCV(LogisticRegression(), param_grid)
logreg.fit(X_train, Y_train)

#print(logreg.cv_results_[{'mean_test_score','std_test_score','params'}])
scoring = logreg.cv_results_
for mean_score, std, params in zip(scoring['mean_test_score'],scoring['std_test_score'],scoring['params']):
    print("{:0.3f} (+/-{:0.03f}) for {}".format(
            mean_score, std * 2, params))


   

In [ ]:
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

# Perform grid search
param_grid = [
    {'C': [1, 10, 100, 1000, 1e4, 1e5, 1e6, 1e7], 'penalty': ['l1', 'l2']}
]
logreg = GridSearchCV(LogisticRegression(), param_grid)
logreg.fit(X_train, Y_train)

# Print grid search results
print('Grid search mean and stdev:\n')
scoring = logreg.cv_results_
for mean_score, std, params in zip(scoring['mean_test_score'],scoring['std_test_score'],scoring['params']):
    print("{:0.3f} (+/-{:0.03f}) for {}".format(
            mean_score, std * 2, params))


# Print best params
print('\nBest parameters:', logreg.best_params_)

print('\nClassification report ({}):\n'.format(key))
print(classification_report(Y_test, logreg.predict(X_test)))